In [143]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
import random
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains


In [2]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    # options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [3]:
def get_name():
    name_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[2]/span'):
        name_all.append(i.text)
        
    return(name_all)

In [4]:
def get_complete():
    complete=[]
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/div[1]/img[1]'):
        if i.get_attribute('alt') == '완강':
            complete.append('O')
        else :
            complete.append('X')
    return(complete)

In [5]:
def get_title():
    title_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/a[1]'):
        title_all.append(i.text)
        
    return(title_all)

In [6]:
def get_link():
    link_all = []
    for i in driver.find_elements_by_xpath('//*[@id="divL_AJAX"]/ul/li[*]/div/div/div[3]/a[1]'):
        link_all.append(i.get_attribute('href'))
        
    return(link_all)

In [7]:
def get_lec_id(link_all):
    lec_id_all=[]
    for i in link_all:
        lec_id_all.append(re.split('=',i)[1])
        
    return(lec_id_all)

In [166]:
def get_data_2(link_all):
    data_2=[]
    num=0
    for link in link_all:
        
        time.sleep(random.random())
        
        num=num+1
        print('강의 구성 수집 중',num,'/',len(link_all))
        table=pd.read_html(link)

        for i in table:
            if list(i.columns)[0]!='강의명':
                continue
            else:
                table=i
                break

        if type(table)==list:
            data_2.extend(['강의 업로드 없음'])
            continue



        table=table.iloc[:,0:2]
        try:
            table = table[table.iloc[:,1].str.contains('[0-9]분')]

            list1 = list(range(1,len(table)+1))
            list2 = list(table.iloc[:,0])
            list3 = list(table.iloc[:,1])
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])
        except:

            index_value= list(table.loc[pd.isnull(table).any(1), :].index.values)


            table = table.drop(index_value)

            table = table[table.iloc[:,1].str.contains('[0-9]분')]

            list1 = list(range(1,len(table)+1))
            list2 = list(table.iloc[:,0])
            list3 = list(table.iloc[:,1])
            data_2.append([','.join(['(' + str(a) + ', ' + b+')' for a,b in zip(list1,list2)])])

    return(data_2)

In [9]:
def get_tch_id(link_all):
    tch_id =[]
    num=0
    for i in link_all:
        num=num+1
        print('teacher_id 수집 중',num,'/',len(link_all))
        res= requests.get(i)
        soup=bs(res.text, 'html.parser')
        tch_id.append(re.split('=',soup.find('span',{'class':'side'}).find('a')['href'])[1])
    return(tch_id)
        

In [10]:
driver = open_chromedriver()

In [11]:
driver.get('https://go3.etoos.com/lecture/mh_list.asp')

In [12]:
name_all = []
complete = []
title_all=[]
link_all =[]
sub_all = []
num=0
len_xpath = len(driver.find_elements_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[*]/li[*]/a'))
for i in range(1,3):
    for j in range(1,6):
        if i==2 and j==5:
            break
            
        driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).click()
        num=num+1
        print('이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중',num,'/',len_xpath)
        if i==2 and j==2:
            for k in range(1,3):
                driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).click()
            
                time.sleep(5)
                
            print(driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).text)


            name= get_name()

            name_all.extend(name)
            link_all.extend(get_link())
            complete.extend(get_complete())
            title_all.extend(get_title())
            sub = driver.find_element_by_xpath('//*[@id="divA_U"]/ul/li[%s]/a' %k).text

            sub_all.extend([sub]*len(name))
            
            if k ==2 :
                driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).click()
                
                time.sleep(1.5)
            
        else:
            
            time.sleep(5)

            
            print(driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).text)

            name= get_name()

            name_all.extend(name)
            link_all.extend(get_link())
            complete.extend(get_complete())
            title_all.extend(get_title())
            sub = driver.find_element_by_xpath('//*[@id="divAJAX_AREA"]/div/ul[%s]/li[%s]/a' %(i,j)).text

            sub_all.extend([sub]*len(name))

이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 1 / 9
국어
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 2 / 9
수학
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 3 / 9
영어
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 4 / 9
사회
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 5 / 9
과학
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 6 / 9
한국사
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 7 / 9
학생부종합
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 8 / 9
제2외/한
이름, 완강 여부, 강의 제목, 강의 링크, 과목 수집 중 9 / 9
인문교양


In [167]:
data_2 = get_data_2(link_all)

강의 구성 수집 중 1 / 2191
강의 구성 수집 중 2 / 2191
강의 구성 수집 중 3 / 2191
강의 구성 수집 중 4 / 2191
강의 구성 수집 중 5 / 2191
강의 구성 수집 중 6 / 2191
강의 구성 수집 중 7 / 2191
강의 구성 수집 중 8 / 2191
강의 구성 수집 중 9 / 2191
강의 구성 수집 중 10 / 2191
강의 구성 수집 중 11 / 2191
강의 구성 수집 중 12 / 2191
강의 구성 수집 중 13 / 2191
강의 구성 수집 중 14 / 2191
강의 구성 수집 중 15 / 2191
강의 구성 수집 중 16 / 2191
강의 구성 수집 중 17 / 2191
강의 구성 수집 중 18 / 2191
강의 구성 수집 중 19 / 2191
강의 구성 수집 중 20 / 2191
강의 구성 수집 중 21 / 2191
강의 구성 수집 중 22 / 2191
강의 구성 수집 중 23 / 2191
강의 구성 수집 중 24 / 2191
강의 구성 수집 중 25 / 2191
강의 구성 수집 중 26 / 2191
강의 구성 수집 중 27 / 2191
강의 구성 수집 중 28 / 2191
강의 구성 수집 중 29 / 2191
강의 구성 수집 중 30 / 2191
강의 구성 수집 중 31 / 2191
강의 구성 수집 중 32 / 2191
강의 구성 수집 중 33 / 2191
강의 구성 수집 중 34 / 2191
강의 구성 수집 중 35 / 2191
강의 구성 수집 중 36 / 2191
강의 구성 수집 중 37 / 2191
강의 구성 수집 중 38 / 2191
강의 구성 수집 중 39 / 2191
강의 구성 수집 중 40 / 2191
강의 구성 수집 중 41 / 2191
강의 구성 수집 중 42 / 2191
강의 구성 수집 중 43 / 2191
강의 구성 수집 중 44 / 2191
강의 구성 수집 중 45 / 2191
강의 구성 수집 중 46 / 2191
강의 구성 수집 중 47 / 2191
강의 구성 수집 중 48 / 2191
강

강의 구성 수집 중 379 / 2191
강의 구성 수집 중 380 / 2191
강의 구성 수집 중 381 / 2191
강의 구성 수집 중 382 / 2191
강의 구성 수집 중 383 / 2191
강의 구성 수집 중 384 / 2191
강의 구성 수집 중 385 / 2191
강의 구성 수집 중 386 / 2191
강의 구성 수집 중 387 / 2191
강의 구성 수집 중 388 / 2191
강의 구성 수집 중 389 / 2191
강의 구성 수집 중 390 / 2191
강의 구성 수집 중 391 / 2191
강의 구성 수집 중 392 / 2191
강의 구성 수집 중 393 / 2191
강의 구성 수집 중 394 / 2191
강의 구성 수집 중 395 / 2191
강의 구성 수집 중 396 / 2191
강의 구성 수집 중 397 / 2191
강의 구성 수집 중 398 / 2191
강의 구성 수집 중 399 / 2191
강의 구성 수집 중 400 / 2191
강의 구성 수집 중 401 / 2191
강의 구성 수집 중 402 / 2191
강의 구성 수집 중 403 / 2191
강의 구성 수집 중 404 / 2191
강의 구성 수집 중 405 / 2191
강의 구성 수집 중 406 / 2191
강의 구성 수집 중 407 / 2191
강의 구성 수집 중 408 / 2191
강의 구성 수집 중 409 / 2191
강의 구성 수집 중 410 / 2191
강의 구성 수집 중 411 / 2191
강의 구성 수집 중 412 / 2191
강의 구성 수집 중 413 / 2191
강의 구성 수집 중 414 / 2191
강의 구성 수집 중 415 / 2191
강의 구성 수집 중 416 / 2191
강의 구성 수집 중 417 / 2191
강의 구성 수집 중 418 / 2191
강의 구성 수집 중 419 / 2191
강의 구성 수집 중 420 / 2191
강의 구성 수집 중 421 / 2191
강의 구성 수집 중 422 / 2191
강의 구성 수집 중 423 / 2191
강의 구성 수집 중

강의 구성 수집 중 752 / 2191
강의 구성 수집 중 753 / 2191
강의 구성 수집 중 754 / 2191
강의 구성 수집 중 755 / 2191
강의 구성 수집 중 756 / 2191
강의 구성 수집 중 757 / 2191
강의 구성 수집 중 758 / 2191
강의 구성 수집 중 759 / 2191
강의 구성 수집 중 760 / 2191
강의 구성 수집 중 761 / 2191
강의 구성 수집 중 762 / 2191
강의 구성 수집 중 763 / 2191
강의 구성 수집 중 764 / 2191
강의 구성 수집 중 765 / 2191
강의 구성 수집 중 766 / 2191
강의 구성 수집 중 767 / 2191
강의 구성 수집 중 768 / 2191
강의 구성 수집 중 769 / 2191
강의 구성 수집 중 770 / 2191
강의 구성 수집 중 771 / 2191
강의 구성 수집 중 772 / 2191
강의 구성 수집 중 773 / 2191
강의 구성 수집 중 774 / 2191
강의 구성 수집 중 775 / 2191
강의 구성 수집 중 776 / 2191
강의 구성 수집 중 777 / 2191
강의 구성 수집 중 778 / 2191
강의 구성 수집 중 779 / 2191
강의 구성 수집 중 780 / 2191
강의 구성 수집 중 781 / 2191
강의 구성 수집 중 782 / 2191
강의 구성 수집 중 783 / 2191
강의 구성 수집 중 784 / 2191
강의 구성 수집 중 785 / 2191
강의 구성 수집 중 786 / 2191
강의 구성 수집 중 787 / 2191
강의 구성 수집 중 788 / 2191
강의 구성 수집 중 789 / 2191
강의 구성 수집 중 790 / 2191
강의 구성 수집 중 791 / 2191
강의 구성 수집 중 792 / 2191
강의 구성 수집 중 793 / 2191
강의 구성 수집 중 794 / 2191
강의 구성 수집 중 795 / 2191
강의 구성 수집 중 796 / 2191
강의 구성 수집 중

강의 구성 수집 중 1119 / 2191
강의 구성 수집 중 1120 / 2191
강의 구성 수집 중 1121 / 2191
강의 구성 수집 중 1122 / 2191
강의 구성 수집 중 1123 / 2191
강의 구성 수집 중 1124 / 2191
강의 구성 수집 중 1125 / 2191
강의 구성 수집 중 1126 / 2191
강의 구성 수집 중 1127 / 2191
강의 구성 수집 중 1128 / 2191
강의 구성 수집 중 1129 / 2191
강의 구성 수집 중 1130 / 2191
강의 구성 수집 중 1131 / 2191
강의 구성 수집 중 1132 / 2191
강의 구성 수집 중 1133 / 2191
강의 구성 수집 중 1134 / 2191
강의 구성 수집 중 1135 / 2191
강의 구성 수집 중 1136 / 2191
강의 구성 수집 중 1137 / 2191
강의 구성 수집 중 1138 / 2191
강의 구성 수집 중 1139 / 2191
강의 구성 수집 중 1140 / 2191
강의 구성 수집 중 1141 / 2191
강의 구성 수집 중 1142 / 2191
강의 구성 수집 중 1143 / 2191
강의 구성 수집 중 1144 / 2191
강의 구성 수집 중 1145 / 2191
강의 구성 수집 중 1146 / 2191
강의 구성 수집 중 1147 / 2191
강의 구성 수집 중 1148 / 2191
강의 구성 수집 중 1149 / 2191
강의 구성 수집 중 1150 / 2191
강의 구성 수집 중 1151 / 2191
강의 구성 수집 중 1152 / 2191
강의 구성 수집 중 1153 / 2191
강의 구성 수집 중 1154 / 2191
강의 구성 수집 중 1155 / 2191
강의 구성 수집 중 1156 / 2191
강의 구성 수집 중 1157 / 2191
강의 구성 수집 중 1158 / 2191
강의 구성 수집 중 1159 / 2191
강의 구성 수집 중 1160 / 2191
강의 구성 수집 중 1161 / 2191
강의 구성 수집 중 

강의 구성 수집 중 1476 / 2191
강의 구성 수집 중 1477 / 2191
강의 구성 수집 중 1478 / 2191
강의 구성 수집 중 1479 / 2191
강의 구성 수집 중 1480 / 2191
강의 구성 수집 중 1481 / 2191
강의 구성 수집 중 1482 / 2191
강의 구성 수집 중 1483 / 2191
강의 구성 수집 중 1484 / 2191
강의 구성 수집 중 1485 / 2191
강의 구성 수집 중 1486 / 2191
강의 구성 수집 중 1487 / 2191
강의 구성 수집 중 1488 / 2191
강의 구성 수집 중 1489 / 2191
강의 구성 수집 중 1490 / 2191
강의 구성 수집 중 1491 / 2191
강의 구성 수집 중 1492 / 2191
강의 구성 수집 중 1493 / 2191
강의 구성 수집 중 1494 / 2191
강의 구성 수집 중 1495 / 2191
강의 구성 수집 중 1496 / 2191
강의 구성 수집 중 1497 / 2191
강의 구성 수집 중 1498 / 2191
강의 구성 수집 중 1499 / 2191
강의 구성 수집 중 1500 / 2191
강의 구성 수집 중 1501 / 2191
강의 구성 수집 중 1502 / 2191
강의 구성 수집 중 1503 / 2191
강의 구성 수집 중 1504 / 2191
강의 구성 수집 중 1505 / 2191
강의 구성 수집 중 1506 / 2191
강의 구성 수집 중 1507 / 2191
강의 구성 수집 중 1508 / 2191
강의 구성 수집 중 1509 / 2191
강의 구성 수집 중 1510 / 2191
강의 구성 수집 중 1511 / 2191
강의 구성 수집 중 1512 / 2191
강의 구성 수집 중 1513 / 2191
강의 구성 수집 중 1514 / 2191
강의 구성 수집 중 1515 / 2191
강의 구성 수집 중 1516 / 2191
강의 구성 수집 중 1517 / 2191
강의 구성 수집 중 1518 / 2191
강의 구성 수집 중 

강의 구성 수집 중 1833 / 2191
강의 구성 수집 중 1834 / 2191
강의 구성 수집 중 1835 / 2191
강의 구성 수집 중 1836 / 2191
강의 구성 수집 중 1837 / 2191
강의 구성 수집 중 1838 / 2191
강의 구성 수집 중 1839 / 2191
강의 구성 수집 중 1840 / 2191
강의 구성 수집 중 1841 / 2191
강의 구성 수집 중 1842 / 2191
강의 구성 수집 중 1843 / 2191
강의 구성 수집 중 1844 / 2191
강의 구성 수집 중 1845 / 2191
강의 구성 수집 중 1846 / 2191
강의 구성 수집 중 1847 / 2191
강의 구성 수집 중 1848 / 2191
강의 구성 수집 중 1849 / 2191
강의 구성 수집 중 1850 / 2191
강의 구성 수집 중 1851 / 2191
강의 구성 수집 중 1852 / 2191
강의 구성 수집 중 1853 / 2191
강의 구성 수집 중 1854 / 2191
강의 구성 수집 중 1855 / 2191
강의 구성 수집 중 1856 / 2191
강의 구성 수집 중 1857 / 2191
강의 구성 수집 중 1858 / 2191
강의 구성 수집 중 1859 / 2191
강의 구성 수집 중 1860 / 2191
강의 구성 수집 중 1861 / 2191
강의 구성 수집 중 1862 / 2191
강의 구성 수집 중 1863 / 2191
강의 구성 수집 중 1864 / 2191
강의 구성 수집 중 1865 / 2191
강의 구성 수집 중 1866 / 2191
강의 구성 수집 중 1867 / 2191
강의 구성 수집 중 1868 / 2191
강의 구성 수집 중 1869 / 2191
강의 구성 수집 중 1870 / 2191
강의 구성 수집 중 1871 / 2191
강의 구성 수집 중 1872 / 2191
강의 구성 수집 중 1873 / 2191
강의 구성 수집 중 1874 / 2191
강의 구성 수집 중 1875 / 2191
강의 구성 수집 중 

강의 구성 수집 중 2190 / 2191
강의 구성 수집 중 2191 / 2191


In [168]:
tch_id=get_tch_id(link_all)

teacher_id 수집 중 1 / 2191
teacher_id 수집 중 2 / 2191
teacher_id 수집 중 3 / 2191
teacher_id 수집 중 4 / 2191
teacher_id 수집 중 5 / 2191
teacher_id 수집 중 6 / 2191
teacher_id 수집 중 7 / 2191
teacher_id 수집 중 8 / 2191
teacher_id 수집 중 9 / 2191
teacher_id 수집 중 10 / 2191
teacher_id 수집 중 11 / 2191
teacher_id 수집 중 12 / 2191
teacher_id 수집 중 13 / 2191
teacher_id 수집 중 14 / 2191
teacher_id 수집 중 15 / 2191
teacher_id 수집 중 16 / 2191
teacher_id 수집 중 17 / 2191
teacher_id 수집 중 18 / 2191
teacher_id 수집 중 19 / 2191
teacher_id 수집 중 20 / 2191
teacher_id 수집 중 21 / 2191
teacher_id 수집 중 22 / 2191
teacher_id 수집 중 23 / 2191
teacher_id 수집 중 24 / 2191
teacher_id 수집 중 25 / 2191
teacher_id 수집 중 26 / 2191
teacher_id 수집 중 27 / 2191
teacher_id 수집 중 28 / 2191
teacher_id 수집 중 29 / 2191
teacher_id 수집 중 30 / 2191
teacher_id 수집 중 31 / 2191
teacher_id 수집 중 32 / 2191
teacher_id 수집 중 33 / 2191
teacher_id 수집 중 34 / 2191
teacher_id 수집 중 35 / 2191
teacher_id 수집 중 36 / 2191
teacher_id 수집 중 37 / 2191
teacher_id 수집 중 38 / 2191
teacher_id 수집 중 39 / 

teacher_id 수집 중 309 / 2191
teacher_id 수집 중 310 / 2191
teacher_id 수집 중 311 / 2191
teacher_id 수집 중 312 / 2191
teacher_id 수집 중 313 / 2191
teacher_id 수집 중 314 / 2191
teacher_id 수집 중 315 / 2191
teacher_id 수집 중 316 / 2191
teacher_id 수집 중 317 / 2191
teacher_id 수집 중 318 / 2191
teacher_id 수집 중 319 / 2191
teacher_id 수집 중 320 / 2191
teacher_id 수집 중 321 / 2191
teacher_id 수집 중 322 / 2191
teacher_id 수집 중 323 / 2191
teacher_id 수집 중 324 / 2191
teacher_id 수집 중 325 / 2191
teacher_id 수집 중 326 / 2191
teacher_id 수집 중 327 / 2191
teacher_id 수집 중 328 / 2191
teacher_id 수집 중 329 / 2191
teacher_id 수집 중 330 / 2191
teacher_id 수집 중 331 / 2191
teacher_id 수집 중 332 / 2191
teacher_id 수집 중 333 / 2191
teacher_id 수집 중 334 / 2191
teacher_id 수집 중 335 / 2191
teacher_id 수집 중 336 / 2191
teacher_id 수집 중 337 / 2191
teacher_id 수집 중 338 / 2191
teacher_id 수집 중 339 / 2191
teacher_id 수집 중 340 / 2191
teacher_id 수집 중 341 / 2191
teacher_id 수집 중 342 / 2191
teacher_id 수집 중 343 / 2191
teacher_id 수집 중 344 / 2191
teacher_id 수집 중 345 / 2191
t

teacher_id 수집 중 613 / 2191
teacher_id 수집 중 614 / 2191
teacher_id 수집 중 615 / 2191
teacher_id 수집 중 616 / 2191
teacher_id 수집 중 617 / 2191
teacher_id 수집 중 618 / 2191
teacher_id 수집 중 619 / 2191
teacher_id 수집 중 620 / 2191
teacher_id 수집 중 621 / 2191
teacher_id 수집 중 622 / 2191
teacher_id 수집 중 623 / 2191
teacher_id 수집 중 624 / 2191
teacher_id 수집 중 625 / 2191
teacher_id 수집 중 626 / 2191
teacher_id 수집 중 627 / 2191
teacher_id 수집 중 628 / 2191
teacher_id 수집 중 629 / 2191
teacher_id 수집 중 630 / 2191
teacher_id 수집 중 631 / 2191
teacher_id 수집 중 632 / 2191
teacher_id 수집 중 633 / 2191
teacher_id 수집 중 634 / 2191
teacher_id 수집 중 635 / 2191
teacher_id 수집 중 636 / 2191
teacher_id 수집 중 637 / 2191
teacher_id 수집 중 638 / 2191
teacher_id 수집 중 639 / 2191
teacher_id 수집 중 640 / 2191
teacher_id 수집 중 641 / 2191
teacher_id 수집 중 642 / 2191
teacher_id 수집 중 643 / 2191
teacher_id 수집 중 644 / 2191
teacher_id 수집 중 645 / 2191
teacher_id 수집 중 646 / 2191
teacher_id 수집 중 647 / 2191
teacher_id 수집 중 648 / 2191
teacher_id 수집 중 649 / 2191
t

teacher_id 수집 중 917 / 2191
teacher_id 수집 중 918 / 2191
teacher_id 수집 중 919 / 2191
teacher_id 수집 중 920 / 2191
teacher_id 수집 중 921 / 2191
teacher_id 수집 중 922 / 2191
teacher_id 수집 중 923 / 2191
teacher_id 수집 중 924 / 2191
teacher_id 수집 중 925 / 2191
teacher_id 수집 중 926 / 2191
teacher_id 수집 중 927 / 2191
teacher_id 수집 중 928 / 2191
teacher_id 수집 중 929 / 2191
teacher_id 수집 중 930 / 2191
teacher_id 수집 중 931 / 2191
teacher_id 수집 중 932 / 2191
teacher_id 수집 중 933 / 2191
teacher_id 수집 중 934 / 2191
teacher_id 수집 중 935 / 2191
teacher_id 수집 중 936 / 2191
teacher_id 수집 중 937 / 2191
teacher_id 수집 중 938 / 2191
teacher_id 수집 중 939 / 2191
teacher_id 수집 중 940 / 2191
teacher_id 수집 중 941 / 2191
teacher_id 수집 중 942 / 2191
teacher_id 수집 중 943 / 2191
teacher_id 수집 중 944 / 2191
teacher_id 수집 중 945 / 2191
teacher_id 수집 중 946 / 2191
teacher_id 수집 중 947 / 2191
teacher_id 수집 중 948 / 2191
teacher_id 수집 중 949 / 2191
teacher_id 수집 중 950 / 2191
teacher_id 수집 중 951 / 2191
teacher_id 수집 중 952 / 2191
teacher_id 수집 중 953 / 2191
t

teacher_id 수집 중 1213 / 2191
teacher_id 수집 중 1214 / 2191
teacher_id 수집 중 1215 / 2191
teacher_id 수집 중 1216 / 2191
teacher_id 수집 중 1217 / 2191
teacher_id 수집 중 1218 / 2191
teacher_id 수집 중 1219 / 2191
teacher_id 수집 중 1220 / 2191
teacher_id 수집 중 1221 / 2191
teacher_id 수집 중 1222 / 2191
teacher_id 수집 중 1223 / 2191
teacher_id 수집 중 1224 / 2191
teacher_id 수집 중 1225 / 2191
teacher_id 수집 중 1226 / 2191
teacher_id 수집 중 1227 / 2191
teacher_id 수집 중 1228 / 2191
teacher_id 수집 중 1229 / 2191
teacher_id 수집 중 1230 / 2191
teacher_id 수집 중 1231 / 2191
teacher_id 수집 중 1232 / 2191
teacher_id 수집 중 1233 / 2191
teacher_id 수집 중 1234 / 2191
teacher_id 수집 중 1235 / 2191
teacher_id 수집 중 1236 / 2191
teacher_id 수집 중 1237 / 2191
teacher_id 수집 중 1238 / 2191
teacher_id 수집 중 1239 / 2191
teacher_id 수집 중 1240 / 2191
teacher_id 수집 중 1241 / 2191
teacher_id 수집 중 1242 / 2191
teacher_id 수집 중 1243 / 2191
teacher_id 수집 중 1244 / 2191
teacher_id 수집 중 1245 / 2191
teacher_id 수집 중 1246 / 2191
teacher_id 수집 중 1247 / 2191
teacher_id 수집 중 1248

teacher_id 수집 중 1506 / 2191
teacher_id 수집 중 1507 / 2191
teacher_id 수집 중 1508 / 2191
teacher_id 수집 중 1509 / 2191
teacher_id 수집 중 1510 / 2191
teacher_id 수집 중 1511 / 2191
teacher_id 수집 중 1512 / 2191
teacher_id 수집 중 1513 / 2191
teacher_id 수집 중 1514 / 2191
teacher_id 수집 중 1515 / 2191
teacher_id 수집 중 1516 / 2191
teacher_id 수집 중 1517 / 2191
teacher_id 수집 중 1518 / 2191
teacher_id 수집 중 1519 / 2191
teacher_id 수집 중 1520 / 2191
teacher_id 수집 중 1521 / 2191
teacher_id 수집 중 1522 / 2191
teacher_id 수집 중 1523 / 2191
teacher_id 수집 중 1524 / 2191
teacher_id 수집 중 1525 / 2191
teacher_id 수집 중 1526 / 2191
teacher_id 수집 중 1527 / 2191
teacher_id 수집 중 1528 / 2191
teacher_id 수집 중 1529 / 2191
teacher_id 수집 중 1530 / 2191
teacher_id 수집 중 1531 / 2191
teacher_id 수집 중 1532 / 2191
teacher_id 수집 중 1533 / 2191
teacher_id 수집 중 1534 / 2191
teacher_id 수집 중 1535 / 2191
teacher_id 수집 중 1536 / 2191
teacher_id 수집 중 1537 / 2191
teacher_id 수집 중 1538 / 2191
teacher_id 수집 중 1539 / 2191
teacher_id 수집 중 1540 / 2191
teacher_id 수집 중 1541

teacher_id 수집 중 1799 / 2191
teacher_id 수집 중 1800 / 2191
teacher_id 수집 중 1801 / 2191
teacher_id 수집 중 1802 / 2191
teacher_id 수집 중 1803 / 2191
teacher_id 수집 중 1804 / 2191
teacher_id 수집 중 1805 / 2191
teacher_id 수집 중 1806 / 2191
teacher_id 수집 중 1807 / 2191
teacher_id 수집 중 1808 / 2191
teacher_id 수집 중 1809 / 2191
teacher_id 수집 중 1810 / 2191
teacher_id 수집 중 1811 / 2191
teacher_id 수집 중 1812 / 2191
teacher_id 수집 중 1813 / 2191
teacher_id 수집 중 1814 / 2191
teacher_id 수집 중 1815 / 2191
teacher_id 수집 중 1816 / 2191
teacher_id 수집 중 1817 / 2191
teacher_id 수집 중 1818 / 2191
teacher_id 수집 중 1819 / 2191
teacher_id 수집 중 1820 / 2191
teacher_id 수집 중 1821 / 2191
teacher_id 수집 중 1822 / 2191
teacher_id 수집 중 1823 / 2191
teacher_id 수집 중 1824 / 2191
teacher_id 수집 중 1825 / 2191
teacher_id 수집 중 1826 / 2191
teacher_id 수집 중 1827 / 2191
teacher_id 수집 중 1828 / 2191
teacher_id 수집 중 1829 / 2191
teacher_id 수집 중 1830 / 2191
teacher_id 수집 중 1831 / 2191
teacher_id 수집 중 1832 / 2191
teacher_id 수집 중 1833 / 2191
teacher_id 수집 중 1834

teacher_id 수집 중 2092 / 2191
teacher_id 수집 중 2093 / 2191
teacher_id 수집 중 2094 / 2191
teacher_id 수집 중 2095 / 2191
teacher_id 수집 중 2096 / 2191
teacher_id 수집 중 2097 / 2191
teacher_id 수집 중 2098 / 2191
teacher_id 수집 중 2099 / 2191
teacher_id 수집 중 2100 / 2191
teacher_id 수집 중 2101 / 2191
teacher_id 수집 중 2102 / 2191
teacher_id 수집 중 2103 / 2191
teacher_id 수집 중 2104 / 2191
teacher_id 수집 중 2105 / 2191
teacher_id 수집 중 2106 / 2191
teacher_id 수집 중 2107 / 2191
teacher_id 수집 중 2108 / 2191
teacher_id 수집 중 2109 / 2191
teacher_id 수집 중 2110 / 2191
teacher_id 수집 중 2111 / 2191
teacher_id 수집 중 2112 / 2191
teacher_id 수집 중 2113 / 2191
teacher_id 수집 중 2114 / 2191
teacher_id 수집 중 2115 / 2191
teacher_id 수집 중 2116 / 2191
teacher_id 수집 중 2117 / 2191
teacher_id 수집 중 2118 / 2191
teacher_id 수집 중 2119 / 2191
teacher_id 수집 중 2120 / 2191
teacher_id 수집 중 2121 / 2191
teacher_id 수집 중 2122 / 2191
teacher_id 수집 중 2123 / 2191
teacher_id 수집 중 2124 / 2191
teacher_id 수집 중 2125 / 2191
teacher_id 수집 중 2126 / 2191
teacher_id 수집 중 2127

In [174]:
lec_id = get_lec_id(link_all)

In [178]:
data = pd.DataFrame({'사이트':['이투스']*len(sub_all),'과목':sub_all,'선생님':name_all,'선생님ID':tch_id,'강좌명':title_all,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2,'링크':link_all})

In [181]:
writer = pd.ExcelWriter(r'이투스.xlsx', engine='xlsxwriter', options={'strings_to_urls': True})
data.to_excel(writer)
writer.close()